In [2]:
import json
import re

def clean_data(data):
    cleaned_data = {
        "มหาวิทยาลัย": data["มหาวิทยาลัย"],
        "รายละเอียด": {}
    }

    for branch, programs in data["รายละเอียด"].items():
        # Remove leading numbers from branch names
        clean_branch = re.sub(r"^\d+\.\s*", "", branch)
        cleaned_data["รายละเอียด"][clean_branch] = []

        for program in programs:
            clean_program = {}
            for key, value in program.items():
                if key == "":
                    continue
                elif key == "ค่าใช้จ่าย":
                    match = re.search(r"\d+,\d+", value.replace(",", ""))
                    clean_program[key] = int(match.group().replace(",", "")) if match else "ไม่ทราบ"
                elif key.startswith("รอบ"):
                    match = re.search(r"\d+", value)
                    clean_program[key] = int(match.group()) if match else 0
                else:
                    clean_program[key] = value

            cleaned_data["รายละเอียด"][clean_branch].append(clean_program)

    return cleaned_data

with open("university_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

cleaned_data = clean_data(data)

with open("universities_cleaned_data.json", "w", encoding="utf-8") as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print("Data cleaned and saved to cleaned_data.json")


TypeError: list indices must be integers or slices, not str

In [3]:
import json
import re

# ฟังก์ชันสำหรับนำตัวเลขออกจากชื่อคณะ
def remove_number_from_branch_name(branch_name):
    return re.sub(r'^\d+\.\s*', '', branch_name)

# ฟังก์ชันสำหรับกรอง key-value ที่มีค่าเป็นสตริงว่าง
def filter_empty_values(data):
    return {k: v for k, v in data.items() if v != ""}

# ฟังก์ชันสำหรับแปลงค่าในหัวข้อค่าใช้จ่าย
def extract_first_number(expense):
    match = re.search(r'\d+', expense.replace(",", ""))
    return int(match.group()) if match else "ไม่ทราบ"

# ฟังก์ชันสำหรับแปลงข้อมูลการรับสมัคร
def process_admission_info(admission):
    if "ไม่เปิดรับสมัครในรอบนี้" in admission:
        return 0
    match = re.search(r'\d+', admission)
    return int(match.group()) if match else 0

# โหลดข้อมูล JSON
with open('all_universities_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# ประมวลผลข้อมูล
for university in data['universities']:
    for branch in university['branches']:
        branch['branch_name'] = remove_number_from_branch_name(branch['branch_name'])
        branch = filter_empty_values(branch)
        for program in branch['programs']:
            if 'ค่าใช้จ่าย' in program:
                program['ค่าใช้จ่าย'] = extract_first_number(program['ค่าใช้จ่าย'])
            for round_key, round_value in program.items():
                if "รอบ" in round_key:
                    program[round_key] = process_admission_info(round_value)

# บันทึกข้อมูลที่ปรับปรุงแล้วลงไฟล์ JSON
with open('filtered_data.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print("ข้อมูลที่ปรับปรุงแล้วถูกบันทึกลงในไฟล์ filtered_data.json")


TypeError: list indices must be integers or slices, not str

In [5]:
import json
import re

# ฟังก์ชันสำหรับนำตัวเลขออกจากชื่อคณะ
def remove_number_from_branch_name(branch_name):
    return re.sub(r'^\d+\.\s*', '', branch_name)

# ฟังก์ชันสำหรับกรอง key-value ที่มีค่าเป็นสตริงว่าง
def filter_empty_values(data):
    return {k: v for k, v in data.items() if v != ""}

# ฟังก์ชันสำหรับแปลงค่าในหัวข้อค่าใช้จ่าย
def extract_first_number(expense):
    match = re.search(r'\d+', expense.replace(",", ""))
    return int(match.group()) if match else "ไม่ทราบ"

# ฟังก์ชันสำหรับแปลงข้อมูลการรับสมัคร
def process_admission_info(admission):
    if "ไม่เปิดรับสมัครในรอบนี้" in admission:
        return 0
    match = re.search(r'\d+', admission)
    return int(match.group()) if match else 0

# โหลดข้อมูล JSON
with open('data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# ประมวลผลข้อมูล
for university in data:
    for branch in university['คณะ']:
        branch['ชื่อคณะ'] = remove_number_from_branch_name(branch['ชื่อคณะ'])
        branch['หลักสูตร'] = [filter_empty_values(program) for program in branch['หลักสูตร']]
        for program in branch['หลักสูตร']:
            if 'ค่าใช้จ่าย' in program:
                program['ค่าใช้จ่าย'] = extract_first_number(program['ค่าใช้จ่าย'])
            for round_key in ['รอบ 1 Portfolio', 'รอบ 2 Quota', 'รอบ 3 Admission', 'รอบ 4 Direct Admission']:
                if round_key in program:
                    program[round_key] = process_admission_info(program[round_key])

# บันทึกข้อมูลที่ปรับปรุงแล้วลงไฟล์ JSON
with open('filtered_data.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print("ข้อมูลที่ปรับปรุงแล้วถูกบันทึกลงในไฟล์ filtered_data.json")


FileNotFoundError: [Errno 2] No such file or directory: 'data.json'

In [11]:
import json
import re

def clean_data(data):
    cleaned_data = []

    for university in data:
        cleaned_university = {
            "มหาวิทยาลัย": university["มหาวิทยาลัย"],
            "รายละเอียด": {}
        }

        for branch, programs in university["รายละเอียด"].items():
            # Remove leading numbers from branch names
            clean_branch = re.sub(r"^\d+\.\s*", "", branch)
            cleaned_university["รายละเอียด"][clean_branch] = []

            for program in programs:
                clean_program = {}
                for key, value in program.items():
                    if key == "":
                        continue
                    elif key == "ค่าใช้จ่าย":
                        # Extract only the first numeric value from ค่าใช้จ่าย
                        match = re.search(r"\d{1,3}(?:,\d{3})*(?:\.\d+)?", value.replace(",", ""))
                        clean_program[key] = int(match.group().replace(",", "")) if match else "ไม่ทราบ"
                    elif key.startswith("รอบ"):
                        # Extract only numeric values from acceptance numbers
                        if "ไม่เปิดรับสมัครในรอบนี้" in value:
                            clean_program[key] = 0
                        else:
                            match = re.search(r"\d+", value)
                            clean_program[key] = int(match.group()) if match else 0
                    else:
                        clean_program[key] = value

                # Remove empty values
                clean_program = {k: v for k, v in clean_program.items() if v != ""}
                cleaned_university["รายละเอียด"][clean_branch].append(clean_program)

        cleaned_data.append(cleaned_university)

    return cleaned_data

# Load JSON data
with open("../data/all_universities_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Clean data
cleaned_data = clean_data(data)

# Save cleaned data
with open("../data/cleaned_universities_data.json", "w", encoding="utf-8") as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print("Data cleaned and saved to cleaned_data.json")


Data cleaned and saved to cleaned_data.json
